<a href="https://colab.research.google.com/github/ahmedokka29/ai-repo-assistant/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 1. Ingest and Index Documents


In [1]:
!pip install minsearch python-frontmatter -q

In [2]:
import io
import logging
import zipfile

import frontmatter
import requests

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.

    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name

    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com'
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)

    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))

    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md')
            or filename_lower.endswith('.mdx')):
            continue

        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    zf.close()
    return repository_data


In [4]:
dtc_faq = read_repo_data('DataTalksClub', 'faq')
evidently_docs = read_repo_data('evidentlyai', 'docs')

print(f"FAQ documents: {len(dtc_faq)}")
print(f"Evidently documents: {len(evidently_docs)}")

FAQ documents: 1224
Evidently documents: 95


In [5]:
evidently_docs[45]

{'title': 'LLM regression testing',
 'description': 'How to run regression testing for LLM outputs.',
 'content': 'In this tutorial, you will learn how to perform regression testing for LLM outputs.\n\nYou can compare new and old responses after changing a prompt, model, or anything else in your system. By re-running the same inputs with new parameters, you can spot any significant changes. This helps you push updates with confidence or identify issues to fix.\n\n<Info>\n  **This example uses Evidently Cloud.** You\'ll run evals in Python and upload them. You can also skip the upload and view Reports locally. For self-hosted, replace `CloudWorkspace` with `Workspace`.\n</Info>\n\n# Tutorial scope\n\nHere\'s what we\'ll do:\n\n* **Create a toy dataset**. Build a small Q&A dataset with answers and reference responses.\n\n* **Get new answers**. Imitate generating new answers to the same question.\n\n* **Create and run a Report with Tests**. Compare the answers using LLM-as-a-judge to eval


# 2. Chunking and Intelligent Processing for Data


## 1. Simple Chunking


In [6]:
def sliding_window(seq, size, step):
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk = seq[i:i+size]
        result.append({'start': i, 'chunk': chunk})
        if i + size >= n:
            break

    return result

res = sliding_window(evidently_docs[45]['content'],2000,1000)

In [7]:
len(res)

21

In [8]:
evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)

In [9]:
len(evidently_chunks)

575

## 2. Splitting by Paragraphs and Sections


In [10]:
import re
text = evidently_docs[45]['content']
paragraphs = re.split(r"\n\s*\n", text.strip())

We use `\n\s*\n` regex pattern for splitting:
- `\n`matches a newline
- `\s*` matches zero or more whitespace characters
- `\n` matches another newline
- So `\n\s*\n` matches two newlines with optional whitespace between them


In [11]:
import re

def split_markdown_by_level(text, level=2):
    """
    Split markdown text by a specific header level.

    :param text: Markdown text as a string
    :param level: Header level to split on
    :return: List of sections as strings
    """
    # This regex matches markdown headers
    # For level 2, it matches lines starting with "## "
    header_pattern = r'^(#{' + str(level) + r'} )(.+)$'
    pattern = re.compile(header_pattern, re.MULTILINE)

    # Split and keep the headers
    parts = pattern.split(text)

    sections = []
    for i in range(1, len(parts), 3):
        # We step by 3 because regex.split() with
        # capturing groups returns:
        # [before_match, group1, group2, after_match, ...]
        # here group1 is "## ", group2 is the header text
        header = parts[i] + parts[i+1]  # "## " + "Title"
        header = header.strip()

        # Get the content after this header
        content = ""
        if i+2 < len(parts):
            content = parts[i+2].strip()

        if content:
            section = f'{header}\n\n{content}'
        else:
            section = header
        sections.append(section)

    return sections

In [12]:
sections = split_markdown_by_level(text, level=2)

In [13]:
evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    sections = split_markdown_by_level(doc_content, level=2)
    for section in sections:
        section_doc = doc_copy.copy()
        section_doc['section'] = section
        evidently_chunks.append(section_doc)

## 3 Intelligent Chunking with LLM

In [52]:
from google.colab import userdata
# userdata.get('GOOGLE_API_KEY')


In [54]:
import google.generativeai as genai
import os

from dotenv import load_dotenv

load_dotenv()

try:
    api_key = userdata.get('GOOGLE_API_KEY')
    # api_key = os.getenv("GOOGLE_API_KEY")

    if not api_key:
        raise ValueError("The environment variable GOOGLE_API_KEY was not found.")

    genai.configure(api_key=api_key)
    print("✅ Google AI configured successfully!")

except ValueError as e:
    logger.error(e)

✅ Google AI configured successfully!


In [15]:
def llm(prompt, model='gemini-2.5-flash'):
    """
    Sends a prompt to the Gemini model and returns the response as text.

    Args:
        prompt (str): The text to send to the model.
        model (str): The Gemini model name to use.

    Returns:
        str: The generated text response from the model.
    """
    model_instance = genai.GenerativeModel(model)

    response = model_instance.generate_content(prompt)

    return response.text


In [16]:
prompt_template = """
Split the provided document into logical sections
that make sense for a Q&A system.

Each section should be self-contained and cover
a specific topic or concept.

<DOCUMENT>
{document}
</DOCUMENT>

Use this format:

## Section Name

Section content with all relevant details

---

## Another Section Name

Another section content

---
""".strip()

In [17]:
def intelligent_chunking(text):
    prompt = prompt_template.format(document=text)
    response = llm(prompt)
    sections = response.split('---')
    sections = [s.strip() for s in sections if s.strip()]
    return sections

In [18]:
# from tqdm.auto import tqdm

# evidently_chunks = []

# for doc in tqdm(evidently_docs):
#     doc_copy = doc.copy()
#     doc_content = doc_copy.pop('content')

#     sections = intelligent_chunking(doc_content)
#     for section in sections:
#         section_doc = doc_copy.copy()
#         section_doc['section'] = section
#         evidently_chunks.append(section_doc)

Bonus: you can use this approach for processing the code in your GitHub repository. You can use a variation of the following prompt:

"Summarize the code in plain English. Briefly describe each class and function/method (their purpose and role), then give a short overall summary of how they work together. Avoid low-level details.". Then add both the source code and the summary to your documents.


In [19]:
# evidently_chunks[0]

In [20]:
# evidently_chunks[8]


# 3. Add Search


## 1. Text search

In [21]:
evidently_docs = read_repo_data('evidentlyai', 'docs')

evidently_chunks = []

for doc in evidently_docs:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        chunk.update(doc_copy)
    evidently_chunks.extend(chunks)

In [22]:
from minsearch import Index

index = Index(
    text_fields=["chunk", "title", "description", "filename"],
    keyword_fields=[]
)

index.fit(evidently_chunks)


In [23]:
query = 'What should be in a test dataset for AI evaluation?'
results = index.search(query)
results

[{'start': 0,
  'chunk': 'Retrieval-Augmented Generation (RAG) systems rely on retrieving answers from a knowledge base before generating responses. To evaluate them effectively, you need a test dataset that reflects what the system *should* know.\n\nInstead of manually creating test cases, you can generate them directly from your knowledge source, ensuring accurate and relevant ground truth data.\n\n## Create a RAG test dataset\n\nYou can generate ground truth RAG dataset from your data source.\n\n### 1. Create a Project\n\nIn the Evidently UI, start a new Project or open an existing one.\n\n* Navigate to “Datasets” in the left menu.\n* Click “Generate” and select the “RAG” option.\n\n![](/images/synthetic/synthetic_data_select_method.png)\n\n### 2. Upload your knowledge base\n\nSelect a file containing the information your AI system retrieves from. Supported formats: Markdown (.md), CSV, TXT, PDFs. Choose how many inputs to generate.\n\n![](/images/synthetic/synthetic_data_inputs_exa

In [24]:
dtc_faq = read_repo_data('DataTalksClub', 'faq')

de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
    text_fields=["question", "content"],
    keyword_fields=[]
)

faq_index.fit(de_dtc_faq)

In [25]:
query = 'When i am able to join the course?'
faq_results = faq_index.search(query)
faq_results

[{'id': '9e508f2212',
  'question': 'Course: When does the course start?',
  'sort_order': 1,
  'content': "The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).\n\n- Register before the course starts using this [link](https://airtable.com/shr6oVXeQvSI5HuWD).\n- Join the [course Telegram channel with announcements](https://t.me/dezoomcamp).\n- Don’t forget to register in DataTalks.Club's Slack and join the channel.",
  'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/001_9e508f2212_course-when-does-the-course-start.md'},
 {'id': '3f1424af17',
  'question': 'Course: Can I still join the course after the start date?',
  'sort_order': 3,
  'content': "Yes, even if you don't register, you're still eligible to submit the homework.\n\nBe aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everything for the last minute.",
  'file

## 2. Vector search


In [26]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
record = de_dtc_faq[2]
text = record['question'] + ' ' + record['content']
v_doc = embedding_model.encode(text)

In [28]:
query = 'I just found out about the course. Can I enroll now?'
v_query = embedding_model.encode(query)

In [29]:
similarity = v_query.dot(v_doc)
similarity

np.float32(0.5190933)

In [30]:
from tqdm.auto import tqdm
import numpy as np

faq_embeddings = []

for d in tqdm(de_dtc_faq):
    text = d['question'] + ' ' + d['content']
    v = embedding_model.encode(text)
    faq_embeddings.append(v)

faq_embeddings = np.array(faq_embeddings)


  0%|          | 0/449 [00:00<?, ?it/s]

In [31]:
from minsearch import VectorSearch

faq_vindex = VectorSearch()
faq_vindex.fit(faq_embeddings, de_dtc_faq)


In [32]:
query = 'Can I join the course now?'
q = embedding_model.encode(query)
results = faq_vindex.search(q)


In [33]:
results

[{'id': '3f1424af17',
  'question': 'Course: Can I still join the course after the start date?',
  'sort_order': 3,
  'content': "Yes, even if you don't register, you're still eligible to submit the homework.\n\nBe aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everything for the last minute.",
  'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/003_3f1424af17_course-can-i-still-join-the-course-after-the-start.md'},
 {'id': '068529125b',
  'question': 'Course - Can I follow the course after it finishes?',
  'sort_order': 8,
  'content': 'Yes, we will keep all the materials available, so you can follow the course at your own pace after it finishes.\n\nYou can also continue reviewing the homeworks and prepare for the next cohort. You can also start working on your final capstone project.',
  'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/008_068529125b_course-can-i-follow-the-course-after-i

In [34]:
evidently_embeddings = []

for d in tqdm(evidently_chunks):
    v = embedding_model.encode(d['chunk'])
    evidently_embeddings.append(v)

evidently_embeddings = np.array(evidently_embeddings)

evidently_vindex = VectorSearch()
evidently_vindex.fit(evidently_embeddings, evidently_chunks)


  0%|          | 0/575 [00:00<?, ?it/s]

## 3. Hybrid search


In [35]:
query = 'Can I join the course now?'

text_results = faq_index.search(query, num_results=5)

q = embedding_model.encode(query)
vector_results = faq_vindex.search(q, num_results=5)

final_results = text_results + vector_results

In [36]:
final_results

[{'id': '3f1424af17',
  'question': 'Course: Can I still join the course after the start date?',
  'sort_order': 3,
  'content': "Yes, even if you don't register, you're still eligible to submit the homework.\n\nBe aware, however, that there will be deadlines for turning in homeworks and the final projects. So don't leave everything for the last minute.",
  'filename': 'faq-main/_questions/data-engineering-zoomcamp/general/003_3f1424af17_course-can-i-still-join-the-course-after-the-start.md'},
 {'id': '9e508f2212',
  'question': 'Course: When does the course start?',
  'sort_order': 1,
  'content': "The next cohort starts January 13th, 2025. More info at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html).\n\n- Register before the course starts using this [link](https://airtable.com/shr6oVXeQvSI5HuWD).\n- Join the [course Telegram channel with announcements](https://t.me/dezoomcamp).\n- Don’t forget to register in DataTalks.Club's Slack and join the c

## Putting this together

In [37]:
def text_search(query):
    return faq_index.search(query, num_results=5)

def vector_search(query):
    q = embedding_model.encode(query)
    return faq_vindex.search(q, num_results=5)

def hybrid_search(query):
    text_results = text_search(query)
    vector_results = vector_search(query)

    # Combine and deduplicate results
    seen_ids = set()
    combined_results = []

    for result in text_results + vector_results:
        if result['filename'] not in seen_ids:
            seen_ids.add(result['filename'])
            combined_results.append(result)

    return combined_results


# Agents and Tools

## 1. Tools and Agents

In [55]:
import google.generativeai as genai

# Create the model
model = genai.GenerativeModel('gemini-2.5-flash')

user_prompt = "I just discovered the course, can I join now?"

# Send the message
response = model.generate_content(user_prompt)

print(response.text)

That's great you're interested in the course!

To give you the most accurate answer, I need a little more information. Could you please tell me:

1.  **What is the name of the course?**
2.  **Where did you discover it (e.g., Coursera, edX, a university website, a specific platform, an independent creator)?**

In general, here's how it usually works:

*   **Self-Paced Courses:** If it's a self-paced course (like many on platforms such as Coursera, Udemy, edX, Khan Academy), you can almost always join and start immediately.
*   **Cohort-Based Courses:** If it's a course with fixed start dates, live sessions, or a specific cohort model (common for university courses, bootcamps, or more intensive programs), then it depends:
    *   **If a cohort has already started:** You might need to wait for the next enrollment period or the next scheduled start date.
    *   **If registration is currently open for an upcoming cohort:** You could likely join now for a future start.
    *   **If registra

## 2. Function Calling with Gemini


In [56]:
text_search_tool = genai.protos.Tool(
    function_declarations=[
        genai.protos.FunctionDeclaration(
            name='text_search',
            description='Search the FAQ database',
            parameters=genai.protos.Schema(
                type=genai.protos.Type.OBJECT,
                properties={
                    'query': genai.protos.Schema(
                        type=genai.protos.Type.STRING,
                        description='Search query text to look up in the course FAQ.'
                    )
                },
                required=['query']
            )
        )
    ])

In [57]:
system_prompt = """
You are a helpful assistant for a course.
"""

question = "I just discovered the course, can I join now?"

# Create model with system instruction and tools
model = genai.GenerativeModel(
    'gemini-2.5-flash',
    system_instruction=system_prompt,
    tools=[text_search_tool]
)

chat = model.start_chat()
response = chat.send_message(question)

In [58]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "text_search",
                  "args": {
                    "query": "Can I join the course now?"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 69,
        "candidates_token_count": 21,
        "total_token_count": 148
      },
      "model_version": "gemini-2.5-flash"
    }),
)

In [59]:
function_call = response.candidates[0].content.parts[0].function_call

if function_call:
    # Extract arguments and call the function
    arguments = {key: value for key, value in function_call.args.items()}
    result = text_search(**arguments)

    # Send function response back
    response = chat.send_message(
        genai.protos.Content(
            parts=[genai.protos.Part(
                function_response=genai.protos.FunctionResponse(
                    name='text_search',
                    response={'result': result}
                )
            )]
        )
    )

print(response.text)

Yes, you can join the course now. Even if you don't register, you're still eligible to submit the homework. However, be aware that there will be deadlines for turning in homeworks and the final projects, so make sure to keep up with the schedule.

The next cohort starts January 13th, 2025. You can find more information at [DTC](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html). You can also register using this [link](https://airtable.com/shr6oVXeQvSI5HuWD) and join the [course Telegram channel with announcements](https://t.me/dezoomcamp). Don't forget to register in DataTalks.Club's Slack and join the channel.


## 3. System Prompt: Instructions


In [60]:
# Create model with system instruction and tools
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash',
    system_instruction=system_prompt,
    tools=[text_search_tool]
)

# Start chat and send message
chat = model.start_chat()
response = chat.send_message(question)

**Key points:**

1.   System prompt goes in system_instruction parameter when creating the model
2.   Tools are also configured at model creation time
3. User message is sent via chat.send_message()
4. No need to manually construct a messages array - the chat object handles history automatically

The system prompt is very important: it influences how the agent behaves. This is how we can control what the agent does and how it responds to user questions

In [61]:
system_prompt = """
You are a helpful assistant for a course.

Use the search tool to find relevant information from the course materials before answering questions.

If you can find specific information through search, use it to provide accurate answers.
If the search doesn't return relevant results, let the user know and provide general guidance.
"""



When working with agents, the system prompt becomes one of the most essential variables we can adjust to influence our agent.
For example, if we want the agent to make multiple search queries, we can modify the prompt:


In [62]:
system_prompt = """
You are a helpful assistant for a course.

Always search for relevant information before answering.
If the first search doesn't give you enough information, try different search terms.

Make multiple searches if needed to provide comprehensive answers.
"""


## 4. Pydantic AI


In [63]:
!pip install pydantic-ai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.6/414.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.8/376.8 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [64]:
from typing import List, Any

def text_search(query: str) -> List[Any]:
    """
    Perform a text-based search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of up to 5 search results returned by the FAQ index.
    """
    return faq_index.search(query, num_results=5)

In [66]:
import os
from pydantic_ai import Agent

# Set API key as environment variable
os.environ['GEMINI_API_KEY'] = userdata.get('GOOGLE_API_KEY')

agent = Agent(
    name="faq_agent",
    instructions=system_prompt,
    tools=[text_search],
    model='gemini-2.5-flash'
)

In [67]:
question = "I just discovered the course, can I join now?"

result = await agent.run(user_prompt=question)

We use await because Pydantic AI is asynchronous. If you're not running in Jupyter, you need to use asyncio.run():

In [68]:
# import asyncio

# result = asyncio.run(agent.run(user_prompt=question))

In [69]:
result

AgentRunResult(output="Yes, you can still join the course even after the start date and are eligible to submit homework. However, be aware that there will be deadlines for turning in homework and the final projects, so it's important not to leave everything until the last minute. The next cohort officially starts on January 13th, 2025.")

In [70]:
result.new_messages()

[ModelRequest(parts=[UserPromptPart(content='I just discovered the course, can I join now?', timestamp=datetime.datetime(2025, 11, 24, 13, 19, 29, 939105, tzinfo=datetime.timezone.utc))], instructions="You are a helpful assistant for a course. \n\nAlways search for relevant information before answering. \nIf the first search doesn't give you enough information, try different search terms.\n\nMake multiple searches if needed to provide comprehensive answers.", run_id='90463122-4587-4962-8433-b208da0c056c'),
 ModelResponse(parts=[ToolCallPart(tool_name='text_search', args={'query': 'join course now enrollment deadline'}, tool_call_id='pyd_ai_4bb3a3b09bdd4a718dbc73a501371fea', provider_details={'thought_signature': 'Ct0BAdHtim+FWjYbFqWHT0u6/rgLdfNnpufMJg5eCHfcDqOx6tpDIjM3H7n8DNIK5+4kWym7r8I5lopq2l7IhXaq5eNEGf3kFsYVXHZqO34EoYDODg0Igli0gmVJU4Z3TIvifr1GakMy+R/IMJVAx48CZybvRxl+HsNsSXYSDzR0vbCyBKK5c+uPbq3DLoUF786UfroIkidUWW0YA/qrAnVXCe40qjP3c9QRFeJ4pcumJbOBfBa46rO+/BaoqKN5KkWopEXaJxsTzdzuf2x6d

# Evaluation